In [ ]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUG_FRONTEND"] = "1"
# os.environ["NUMBA_DEBUG"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["NUMBA_SHOW_HELP"] = "1"
# os.environ["NUMBA_FULL_TRACEBACKS"] = "1"
# os.environ["NUMBA_DEVELOPER_MODE"] = "1"

In [ ]:
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_base import backtest_df_only
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import nb_BacktestInd, nb_Strategy
from numba import njit

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [ ]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=200,
)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([3, 4, 5, 6]),
    max_trades=np.array([5, 6]),
    num_candles=np.array([60]),
    risk_account_pct_size=np.array([3, 5, 10]),
    risk_reward=np.array([3]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([10, 20, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1.20]),
    trail_sl_when_pct=np.array([0.001]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = nb_dos_cart_product(
    dos_arrays=dos_arrays,
)
ex_settings = mufex_main.exchange_settings

In [ ]:
from nb_quantfreedom.nb_base import get_classes_yes

(
    backtest_settings,
    candles,
    checker_liq_hit,
    checker_sl_hit,
    checker_sl_to_be,
    checker_tp_hit,
    checker_tsl,
    dec_pos_calculator,
    dos_cart_arrays,
    exchange_settings,
    exit_fee_pct,
    inc_pos_calculator,
    lev_calculator,
    logger,
    set_z_e,
    sl_bcb_price_getter,
    sl_calculator,
    sl_mover,
    starting_equity,
    strategy,
    ind_creator,
    total_bars,
    total_indicator_settings,
    total_order_settings,
    tp_calculator,
) = get_classes_yes(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_cart_arrays=dos_cart_arrays,
    exchange_settings=ex_settings,
    logger_settings=None,
    # logger_settings="p",
    # logger_settings=LoggerSettings(log_debug=True, create_trades_logger=False),
    starting_equity=1000.0,
    static_os=static_os,
    strategy=nb_Strategy(),
    ind_creator=nb_BacktestInd(),
)

In [ ]:
from nb_quantfreedom.nb_simulate import nb_run_backtest


@njit(cache=True)
def testing(
    backtest_settings,
    candles,
    checker_liq_hit,
    checker_sl_hit,
    checker_sl_to_be,
    checker_tp_hit,
    checker_tsl,
    dec_pos_calculator,
    dos_cart_arrays,
    exchange_settings,
    exit_fee_pct,
    inc_pos_calculator,
    lev_calculator,
    logger,
    set_z_e,
    sl_bcb_price_getter,
    sl_calculator,
    sl_mover,
    starting_equity,
    strategy,
    ind_creator,
    total_bars,
    total_indicator_settings,
    total_order_settings,
    tp_calculator,
):
    return nb_run_backtest(
        backtest_settings=backtest_settings,
        candles=candles,
        checker_liq_hit=checker_liq_hit,
        checker_sl_hit=checker_sl_hit,
        checker_sl_to_be=checker_sl_to_be,
        checker_tp_hit=checker_tp_hit,
        checker_tsl=checker_tsl,
        dec_pos_calculator=dec_pos_calculator,
        dos_cart_arrays=dos_cart_arrays,
        exchange_settings=exchange_settings,
        exit_fee_pct=exit_fee_pct,
        inc_pos_calculator=inc_pos_calculator,
        lev_calculator=lev_calculator,
        logger=logger,
        set_z_e=set_z_e,
        sl_bcb_price_getter=sl_bcb_price_getter,
        sl_calculator=sl_calculator,
        sl_mover=sl_mover,
        starting_equity=starting_equity,
        strategy=strategy,
        ind_creator=ind_creator,
        total_bars=total_bars,
        total_indicator_settings=total_indicator_settings,
        total_order_settings=total_order_settings,
        tp_calculator=tp_calculator,
    )

In [ ]:
yes_yes_yes = testing(
    backtest_settings,
    candles,
    checker_liq_hit,
    checker_sl_hit,
    checker_sl_to_be,
    checker_tp_hit,
    checker_tsl,
    dec_pos_calculator,
    dos_cart_arrays,
    exchange_settings,
    exit_fee_pct,
    inc_pos_calculator,
    lev_calculator,
    logger,
    set_z_e,
    sl_bcb_price_getter,
    sl_calculator,
    sl_mover,
    starting_equity,
    strategy,
    ind_creator,
    total_bars,
    total_indicator_settings,
    total_order_settings,
    tp_calculator,
)

In [ ]:
import pandas as pd
pd.DataFrame(yes_yes_yes)

In [ ]:
type(round(10.49859))